<a href="https://colab.research.google.com/github/gencross/CollatzGroup3/blob/main/MilesCollatzNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
def collatz_verbose(n):
  '''
  Prints the orbit of n
  '''
  while n != 1:
    print(n)
    if n % 2 == 0:
      n //= 2
    else:
      n = (3 * n + 1) // 2
  return

In [17]:
def collatz_iterations(n):
  '''
  Return how many iterations it takes for
  n to go to 1
  '''
  iterations = 0
  while n != 1:
    if n % 2 == 0:
      n //= 2
    else:
      n = (3 * n + 1) // 2
    iterations += 1
  return iterations

In [18]:
def collatz_check(n):
  '''
  Returns True if n passes
  the Collatz conjecture
  '''
  while n != 1:
    if n % 2 == 0:
      n //= 2
    else:
      n = (3 * n + 1) // 2
  return True

In [19]:
for i in range(2,10**5):
  if collatz_check(i) != True:
    print("Fields Medal!!!", i)
print("Check complete.")    

Check complete.


In [20]:
def find_max_iter(n):
  '''
  Checks for the number with the longest orbit
  up to n. Returns a tuple. The first value is
  the number, the second value is the number of 
  iterations.
  '''
  max_iter = 0
  max_iter_n = 0
  for i in range(2,n+1):
    i_iter = collatz_iterations(i)
    if i_iter > max_iter:
      max_iter = i_iter
      max_iter_n = i
  return max_iter_n, max_iter

In [21]:
find_max_iter(10**5)

(77031, 221)

In [22]:
collatz_verbose(77031)

77031
115547
173321
259982
129991
194987
292481
438722
219361
329042
164521
246782
123391
185087
277631
416447
624671
937007
1405511
2108267
3162401
4743602
2371801
3557702
1778851
2668277
4002416
2001208
1000604
500302
250151
375227
562841
844262
422131
633197
949796
474898
237449
356174
178087
267131
400697
601046
300523
450785
676178
338089
507134
253567
380351
570527
855791
1283687
1925531
2888297
4332446
2166223
3249335
4874003
7311005
10966508
5483254
2741627
4112441
6168662
3084331
4626497
6939746
3469873
5204810
2602405
3903608
1951804
975902
487951
731927
1097891
1646837
2470256
1235128
617564
308782
154391
231587
347381
521072
260536
130268
65134
32567
48851
73277
109916
54958
27479
41219
61829
92744
46372
23186
11593
17390
8695
13043
19565
29348
14674
7337
11006
5503
8255
12383
18575
27863
41795
62693
94040
47020
23510
11755
17633
26450
13225
19838
9919
14879
22319
33479
50219
75329
112994
56497
84746
42373
63560
31780
15890
7945
11918
5959
8939
13409
20114
10057
15086
7543
